In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
def load_yolo(model_path="yolov8n.pt"):
    return YOLO(model_path)  # Load YOLOv8 Nano (fastest for real-time)

In [ ]:
def init_tracker():
    return DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0)

In [ ]:
def detect_person(frame, model, conf_threshold=0.5):
    results = model(frame)[0]  # Get first batch result
    detections = []

    for box in results.boxes.data:
        x1, y1, x2, y2, conf, class_id = box.tolist()
        if int(class_id) == 0 and conf > conf_threshold:  # Class 0 is 'person'
            detections.append(([x1, y1, x2, y2], conf, class_id))

    return detections

In [ ]:
# Compute Left-Right Score based on the person’s horizontal position
def compute_lr_score(cx, frame_width):
    lr_score = (2 * (cx - frame_width / 2)) / frame_width  # Normalize between -1 and 1
    return round(lr_score, 3)

In [ ]:
def compute_movement_score(area_history, N=10):
    if len(area_history) < 2:
        return 0.5  # Default to stationary

    delta_areas = [area_history[i] - area_history[i - 1] for i in range(1, len(area_history))]
    avg_delta = sum(delta_areas) / len(delta_areas)  # Average rate of change

    # Normalize using sigmoid to keep the value between 0 and 1
    sigmoid = 1 / (1 + np.exp(-avg_delta / max(area_history)))  
    return round(1 - sigmoid, 3)  # Flip it (1 = moving away, 0 = moving closer)

In [ ]:
def track_first_person(frame, tracker, first_person_id, detections):
    tracked_objects = tracker.update_tracks(detections, frame=frame)
    new_id, curr_area, lr_score = None, None, None

    h, w, _ = frame.shape  # Get image width for LR score computation

    for track in tracked_objects:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Assign the first detected ID if not set
        if first_person_id is None:
            first_person_id = track_id

        # Track only the first detected person
        if track_id == first_person_id:
            new_id = first_person_id
            curr_area = (x2 - x1) * (y2 - y1)  # Compute bounding box area

            # Compute horizontal position score
            cx = (x1 + x2) // 2  # Calculate centroid x-coordinate
            lr_score = compute_lr_score(cx, w)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Tracking ID {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame, new_id, curr_area, lr_score

In [ ]:
# Main function to process video
def run_camera():
    model = load_yolo()
    tracker = init_tracker()
    cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam

    first_person_id = None
    area_history = deque(maxlen=5)  # Store last 5 frames' bounding box areas

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Detect people
        detections = detect_person(frame, model)

        # Step 2: Track only the first detected person
        frame, first_person_id, curr_area, lr_score = track_first_person(frame, tracker, first_person_id, detections)

        # Step 3: Compute movement score
        if curr_area:
            area_history.append(curr_area)
            movement_score = compute_movement_score(area_history)

            # Display movement and LR scores
            cv2.putText(frame, f"Movement Score: {movement_score}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            if lr_score is not None:
                cv2.putText(frame, f"LR Score: {lr_score}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 165, 0), 2)

        # Display frame
        cv2.imshow("Person Tracking with Movement & LR Score", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
run_camera()